In [46]:
import ast
import glob

In [49]:
# Opening JSON file
for filepath in glob.iglob('./tweet_jsons/*.json'):
    print(filepath)
    f = open(filepath, "r")
    Lines = f.readlines()
    
    print("\n")
    
    for line in Lines:
        print(json.loads(line)["content"])
        
    print("\n")

./tweet_jsons/pmarca.json


Tonight at 5PM Pacific, join us on @joinClubhouse as @alighodsi interviews @bhorowitz and me on founding and building @a16z! https://t.co/3ypI0qCP3g
I'm discussing “Unfettered Conversations with A &amp; Z featuring Steve Stoute” with Steve J Stoute, @bhorowitz, and a16z. Monday, Apr 5 at 7:00 PM PDT on @joinclubhouse. Join us! https://t.co/zEZ40NzR2x
I'm discussing “Boss Talk” with @bhorowitz, Michael Ovitz, @alighodsi, Felicia Horowitz, and a16z. Tuesday, Apr 6 at 5:00 PM PDT on @joinclubhouse. Join us! https://t.co/plyqJCYfMt


./tweet_jsons/rogerkver.json


Voluntary trade is the foundation of human civilization.
Imagine being so brainwashed that you believe some humans need permission from other humans to transmit money.  #cryptocurrency
https://t.co/6EeRmpfaH7 is not for sale. https://t.co/lKUGfZprox
Better cash makes a better store of value.
Rejecting the religion of Statism is more difficult than other religions because almost everyone currently belie